# Assignment 1

Deadline: 19.03.2025, 12:00 CET

<Add your name, student-id and emal address>

In [4]:
# Import standard libraries
import os
import sys
import timeit # To compute runtimes
from typing import Optional

# Import third-party libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import local modules
project_root = os.path.dirname(os.path.dirname(os.getcwd()))   # Change this path if needed
src_path = os.path.join(project_root, 'qpmwp-course\\src')
sys.path.append(project_root)
sys.path.append(src_path)
from estimation.covariance import Covariance
from estimation.expected_return import ExpectedReturn
from optimization.constraints import Constraints
from optimization.optimization import Optimization, Objective
from optimization.optimization_data import OptimizationData
from optimization.quadratic_program import QuadraticProgram, USABLE_SOLVERS

## 1. Solver Horse Race

### 1.a)
(3 points)

Generate a Multivariate-Normal random dataset of dimension TxN, T=1000, N=100, and compute a vector of expected returns, q, and a covariance matrix, P, using classes ExpectedReturn and Covariance respectively.

In [11]:

# Set the dimensions
T = 1000  # Number of time periods
N = 100   # Number of assets

# Generate a random mean vector from a normal distribution
mean = np.random.rand(N)
# Generate a random covariance matrix
A = np.random.rand(N, N)
cov = np.dot(A, A.transpose())  # To ensure the covariance matrix is positive semi-definite

# Generate the Multivariate-Normal random dataset
data = np.random.multivariate_normal(mean, cov, T)

# Convert the dataset to a DataFrame for easier manipulation
df = pd.DataFrame(data, columns=[f'Asset_{i+1}' for i in range(N)])


# Compute the vector of expected returns (mean returns) from df
scalefactor = 1
expected_return = ExpectedReturn(method='arithmetic', scalefactor=scalefactor)
expected_return.estimate(X=df, inplace=True)

# Compute the covariance matrix from df
covariance = Covariance(method='pearson')
covariance.estimate(X=df, inplace=True)


# Display the results
print("Vector of expected returns (q):")
print(expected_return.vector)
# 
print("\nCovariance matrix (P):")
print(covariance.matrix)

Vector of expected returns (q):
Asset_1      0.685387
Asset_2      0.842107
Asset_3      0.424560
Asset_4      0.749159
Asset_5      0.347585
               ...   
Asset_96     0.981816
Asset_97     0.959098
Asset_98     0.250784
Asset_99     0.784766
Asset_100    0.349733
Length: 100, dtype: float64

Covariance matrix (P):
             Asset_1    Asset_2    Asset_3    Asset_4    Asset_5    Asset_6  \
Asset_1    26.113495  21.374907  22.505923  19.590368  20.891640  20.406451   
Asset_2    21.374907  31.591600  26.193931  23.326526  22.056617  21.375757   
Asset_3    22.505923  26.193931  33.712525  25.545640  24.313666  24.345191   
Asset_4    19.590368  23.326526  25.545640  29.709464  20.726178  18.976139   
Asset_5    20.891640  22.056617  24.313666  20.726178  28.738330  20.418363   
...              ...        ...        ...        ...        ...        ...   
Asset_96   22.135860  21.355930  26.168746  21.463474  23.713311  22.115297   
Asset_97   21.726616  22.295494  22.721469

### 1.b)
(3 points)

Instantiate a constraints object by injecting column names of the data created in 1.a) as ids and add:
- a budget constaint (i.e., asset weights have to sum to one)
- lower bounds of 0.0 for all assets
- upper bounds of 0.2 for all assets
- group contraints such that the sum of the weights of the first 30 assets is <= 0.3, the sum of assets 31 to 60 is <= 0.4 and the sum of assets 61 to 100 is <= 0.5

In [12]:
# Instantiate the Constraints class
constraints = Constraints(ids = df.columns.tolist())

# Add budget constraint
constraints.add_budget(rhs=1, sense='=')

# Add box constraints (i.e., lower and upper bounds)
constraints.add_box(lower=0, upper=0.2)

# Add linear constraints
G = pd.DataFrame(np.zeros((3, N)), columns=constraints.ids)
G.iloc[0, :30] = 1
G.iloc[1, 30:60] = 1
G.iloc[2, 60:] = 1
h = pd.Series([0.3, 0.4, 0.5])
constraints.add_linear(G=G, sense='<=', rhs=h)


### 1.c) 
(4 points)

Solve a Mean-Variance optimization problem (using coefficients P and q in the objective function) which satisfies the above defined constraints.
Repeat the task for all open-source solvers in qpsolvers and compare the results in terms of:

- runtime
- accuracy: value of the primal problem.
- reliability: are all constarints fulfilled? Extract primal resisduals, dual residuals and duality gap.

Generate a DataFrame with the solvers as column names and the following row index: 'solution_found': bool, 'objective': float, 'primal_residual': float, 'dual_residual': float, 'duality_gap': float, 'runtime': float.

Put NA's for solvers that failed for some reason (e.g., unable to install the package or solvers throws an error during execution). 




In [13]:
list(USABLE_SOLVERS)

['highs', 'osqp', 'qpalm', 'daqp', 'cvxopt', 'quadprog']

In [14]:
import time
# Extract the constraints in the format required by the solver
GhAb = constraints.to_GhAb()

# Loop over solvers, instantiate the quadratic program, solve it and store the results
risk_aversion = 1

results = pd.DataFrame(columns=list(USABLE_SOLVERS), index={'solution_found': bool, 'objective': float, 'primal_residual': float, 'dual_residual': float, 'duality_gap': float, 'runtime': float})

for solver_name in USABLE_SOLVERS:
    qp = QuadraticProgram(
        P = covariance.matrix.to_numpy() * risk_aversion,
        q = expected_return.vector.to_numpy() * -1,
        G = GhAb['G'],
        h = GhAb['h'],
        A = GhAb['A'],
        b = GhAb['b'],
        lb = constraints.box['lower'].to_numpy(),
        ub = constraints.box['upper'].to_numpy(),
        solver = solver_name
    )

    start_time = time.time()
    qp.solve()
    end_time = time.time()

    sol = qp.results.get('solution')
    results.loc['solution_found', solver_name] = sol.found
    if sol.found:
        results.loc['objective', solver_name] = qp.objective_value()
        results.loc['primal_residual', solver_name] = sol.primal_residual()
        results.loc['dual_residual', solver_name] = sol.dual_residual()
        results.loc['duality_gap', solver_name] = sol.duality_gap()
        results.loc['runtime', solver_name] = end_time - start_time

Print and visualize the results

In [15]:
print(results)

                                  highs                 osqp  \
solution_found                     True                 True   
objective                      9.642079             9.577749   
primal_residual                     0.0              0.00082   
dual_residual                       0.0             0.001854   
duality_gap      1.0181608445236634e-08  0.06325655098640226   
runtime                        0.017535             0.016056   

                                 qpalm                    daqp  \
solution_found                    True                    True   
objective                     9.641409                9.642079   
primal_residual               0.000022                     0.0   
dual_residual                      0.0                     0.0   
duality_gap      0.0006693867048959135  1.0693668173189508e-12   
runtime                       0.021678                0.008437   

                                 cvxopt               quadprog  
solution_found         

## 2. Analytical Solution to Minimum-Variance Problem

(5 points)

- Create a `MinVariance` class that follows the structure of the `MeanVariance` class.
- Implement the `solve` method in `MinVariance` such that if `solver_name = 'analytical'`, the analytical solution is computed and stored within the object (if such a solution exists). If not, call the `solve` method from the parent class.
- Create a `Constraints` object by injecting the same ids as in part 1.b) and add a budget constraint.
- Instantiate a `MinVariance` object by setting `solver_name = 'analytical'` and passing instances of `Constraints` and `Covariance` as arguments.
- Create an `OptimizationData` object that contains an element `return_series`, which consists of the synthetic data generated in part 1.a).
- Solve the optimization problem using the created `MinVariance` object and compare the results to those obtained in part 1.c).


In [18]:
# Define class MinVariance
from helper_functions import to_numpy

class MinVariance(Optimization):

    def __init__(self,
                 constraints: Constraints,
                 covariance: Optional[Covariance] = None,
                 **kwargs):
        super().__init__(
            constraints=constraints,
            **kwargs
        )
        self.covariance = Covariance() if covariance is None else covariance

    def set_objective(self, optimization_data: OptimizationData) -> None:
        X = optimization_data['return_series']
        covmat = self.covariance.estimate(X=X, inplace=False)
        self.objective = Objective(
            P = covmat
        )
        return None
    
    def solve(self) -> None:
        if self.params.get('solver_name') == 'analytical':
            constraints = self.constraints
            GhAb = constraints.to_GhAb()
            obj_coeff = self.objective.coefficients

            E = to_numpy(obj_coeff['P'])
            A = GhAb['A']
            b = GhAb['b']
            
            try:
                weights = np.linalg.inv(E) @ A.T @ np.linalg.inv(A @ np.linalg.inv(E) @ A.T) @ b
                self.results.update({
                    'weights': weights,
                    'status': True
                })        
                return None
            
            except:
                weights = [0]*len(E)
                self.results.update({
                    'weights': weights,
                    'status': False
                })        
                return None

        else:
            return super().solve()


# Create a constraints object with just a budget constraint
constraints = Constraints(ids = df.columns.tolist())
constraints.add_budget(rhs=[1], sense='=')

# Instantiate the MinVariance class
minvar = MinVariance(
    constraints=constraints,
    covariance=covariance,
    solver_name='analytical'
)

# Prepare the optimization data and prepare the optimization problem
optimization_data = OptimizationData(return_series=df)
minvar.set_objective(optimization_data=optimization_data)

# Solve the optimization problem and print the weights
minvar.solve()
print(minvar.results.get('weights'))

[-8.45886652e-01 -1.73032690e+00 -3.47340153e+00 -6.94895232e-02
 -1.33752904e-01 -5.49375570e-02  9.77318517e+00  7.54549416e+00
  3.99504795e+00  1.75812973e+00 -1.20461434e+00  1.19154199e+00
 -2.82004094e+00 -5.46782990e+00  2.30799022e+00  6.07342208e+00
  4.76325639e+00 -5.31270991e+00 -1.06054788e+01  2.70024264e+00
  2.38552401e+00 -4.11283225e+00 -4.63540911e+00  4.19886029e+00
  1.42663025e+00 -1.06687391e+01 -3.30588249e-01 -1.34605444e-01
 -8.51462311e+00  4.12197669e+00  8.66655015e-01  3.07176643e+00
 -3.45045226e+00  2.86965417e-01  1.16907979e+00 -9.22743036e+00
 -9.38574153e+00 -3.47137035e+00  1.11814452e+01 -4.31901261e-01
 -3.51980386e+00  4.51863903e+00  1.55929217e+00 -8.00083253e+00
 -5.02228021e-01 -3.82416838e+00  1.12312147e-02  4.89504687e+00
  1.07052774e+00 -2.35433421e+00  8.59845666e+00 -3.05807155e+00
  3.44718972e+00  5.33158775e+00  2.17388363e+00  9.99351971e+00
 -9.83726290e+00 -1.70236773e+00 -3.49118447e+00  8.96210204e-01
 -1.97114282e+00 -4.98523